In [9]:
import pandas as pd
from dfply import *

import sys
from sshtunnel import SSHTunnelForwarder
import logging
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')

import src.data_extraction.access_database as db   # DB 연결 관련 모듈
import src.utils.utils as utils   # 기타 유틸리티 모듈

## Vital Sign
- mbp
- ...

### 1. Subjectlist 가져오기

In [5]:
# 데이터 저장위치
# data_dir = './data'   # 수정 쌤 디렉토리에서 사용
data_dir = '../../outputs'   # 내 디렉에서 사용
sub_list_dir = data_dir + '/' + 'sub1list.csv'


subjectlist1 = pd.read_csv(sub_list_dir, parse_dates=['admittime', 'dischtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id', 'admittime', 
                                'dischtime', 'gender', 'age', 'height', 
                                'weight')
sub1list = sub1list.drop_duplicates(subset='stay_id')   # 삽관/발관 포함 안된 데이터임. 순전히 고유 hadm_id, stay_id 만 있음.
sub1list

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8
...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5


### 2. DB 연결 후 필요한 테이블 가져오기
- vitalsign

In [3]:
# 1. DB 파라미터 설정

# 데이터 저장위치
# output_dir = './data'   # 수정 쌤 디렉토리에서 사용
output_dir = '../../outputs'   # 내 디렉에서 사용


# SSH 연결 설정
ssh_config = {
    'ssh_ip': '210.91.223.248',
    'ssh_port': 35430,
    'ssh_username': "medicalai",
    'ssh_password': "medicalai731!",
    'remote_bind_address': ('localhost', 35432)  # 데이터베이스 호스트 및 포트
}

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_admin',
    'password': 'admin1q2w!@',
    'host': 'localhost',  # SSH 터널을 사용하면, 이 값은 'localhost'가 됩니다
    'port': 35432  # 이 값은 SSH 터널 로컬 포트 바인딩에 의해 대체됩니다
}


# 테이블 쿼리 설정
tables_query = {
    'vitalsign': "select * from mimiciv_derived.vitalsign;"
                }


db.print_config_info(db_config, tables_query)

--------- Database Configuration ---------
Database Name: mimiciv
User: mai_admin
Password: ***********
Host: localhost
Port: 35432

--------- SQL Queries for Required Tables ---------
vitalsign: select * from mimiciv_derived.vitalsign;



In [4]:
# 2. db 접속, 테이블 추출, 'dataframes' 라는 딕셔너리에 테이블 저장

try:
    conn, tunnel = db.connect_to_database_via_ssh(db_config, ssh_config)
    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")
except Exception as e:
    logging.error(f"An error occurred: {e}")
finally:
    if conn is not None:
        db.disconnect_database(conn)


# 데이터프레임 변수로 저장
## vitalsign
vitalsign = dataframes['vitalsign']


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:SSH TUNNEL ESTABLISHED...
INFO:root:mimiciv DATABASE CONNECTED VIA SSH.
INFO:root:Retrieved vitalsign: (9745500, 15)
INFO:root:DATABASE CONNCETION CLOSED.


In [7]:
# datetime 변환
time_cols = ['charttime']
vitalsign = utils.to_datetime(vitalsign, time_cols)
print(vitalsign.shape)

Converted charttime to datetime.
(9745500, 15)


In [26]:
vitalsign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9745500 entries, 0 to 9745499
Data columns (total 15 columns):
 #   Column            Dtype         
---  ------            -----         
 0   subject_id        int64         
 1   stay_id           int64         
 2   charttime         datetime64[ns]
 3   heart_rate        float64       
 4   sbp               float64       
 5   dbp               float64       
 6   mbp               float64       
 7   sbp_ni            float64       
 8   dbp_ni            float64       
 9   mbp_ni            float64       
 10  resp_rate         float64       
 11  temperature       object        
 12  temperature_site  object        
 13  spo2              float64       
 14  glucose           float64       
dtypes: datetime64[ns](1), float64(10), int64(2), object(2)
memory usage: 1.1+ GB


### 3. mean blood pressure (MBP) 만 결합

In [27]:
# gender 결합 (subject_id 기준)
sub1list_mbp = pd.merge(sub1list, vitalsign[['subject_id', 'stay_id', 'charttime', 'mbp']], on=['subject_id', 'stay_id'], how='left')
sub1list_mbp

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,mbp
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 04:00:00,NaN
1,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 04:20:00,NaN
2,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 04:22:00,NaN
3,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 04:25:00,46.0
4,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 05:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2267694,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:17:00,44.0
2267695,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:18:00,NaN
2267696,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:20:00,NaN
2267697,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:34:00,NaN


### 4. mbp null 값 제거, 중복값 확인
- 주의: vital 데이터의 경우 완전 중복값이 아닌 근접값도 고려할 필요 있음

In [33]:
# null 값 확인
sub1list_mbp_notna = sub1list_mbp[sub1list_mbp.mbp.notna()]
sub1list_mbp_notna

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,mbp
3,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 04:25:00,46.0
5,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 05:01:00,70.0
7,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 06:01:00,75.0
11,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 06:47:00,84.0
13,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 07:01:00,73.0
...,...,...,...,...,...,...,...,...,...,...,...
2267687,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 12:00:00,65.0
2267688,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:00:00,45.0
2267689,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:05:00,51.0
2267690,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:10:00,33.0


In [34]:
# null 값 제거 후 stay id 카운트 동일.

print(f'stay_id COUNT BEFORE: {sub1list_mbp.stay_id.nunique()}')
print(f'stay_id COUNT AFTER: {sub1list_mbp_notna.stay_id.nunique()}')

stay_id COUNT BEFORE: 7231
stay_id COUNT AFTER: 7231


In [37]:
# 중복값 확인 --> 없음
## 중복값 정의: stay_id 내에서 charttime 동일한 경우
duplicates = sub1list_mbp_notna[sub1list_mbp_notna.duplicated(subset=['stay_id', 'charttime'], keep=False)]
duplicates_sorted = duplicates.sort_values(by=['stay_id', 'charttime'])   # optional

duplicates_sorted


,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,mbp


### 5. hadm_id, stay_id 별로 시간순 (charttime) 정렬

In [38]:
sub1list_mbp_notna.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'charttime'], inplace=True)
sub1list_mbp_notna

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub1list_mbp_notna.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'charttime'], inplace=True)



,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,mbp
3,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 04:25:00,46.0
5,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 05:01:00,70.0
7,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 06:01:00,75.0
11,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 06:47:00,84.0
13,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 07:01:00,73.0
...,...,...,...,...,...,...,...,...,...,...,...
2267687,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 12:00:00,65.0
2267688,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:00:00,45.0
2267689,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:05:00,51.0
2267690,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 13:10:00,33.0


### 6. 데이터 저장

In [39]:
file_name = data_dir + '/' + 'sub1list_mbp.csv'
sub1list_mbp_notna.to_csv(file_name, index=False, encoding='utf-8-sig')